<a href="https://colab.research.google.com/github/ShreyasJothish/DS-Unit-4-Sprint-3-Neural-Networks/blob/master/module1-Intro-to-Neural-Networks/LS_DS_431_Intro_to_NN_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Intro to Neural Networks Assignment

## Define the Following:
You can add image, diagrams, whatever you need to ensure that you understand the concepts below.

### Input Layer:

* The Input Layer is the 1st neuron layer which receives input from our dataset. 
* This is also called the visible layer because it's the only part that is exposed to our data and that our data interacts with directly. 
* Typically node maps within the input layer corresponds with one input node for each of the different inputs/features/columns of our dataset that will be passed to the neural network.

### Hidden Layer:

* The neuron layers between Input layer and Output layer are called Hidden layers since it cannot be accessed directly.
* Weighted sum of inputs and bias from previous layers are passed onto hidden layer for processing.
* There can be multiple hidden layers and usually refers to Deep Learning.

### Output Layer:

* The final layer of Neural Network is called Output layer.
* The purpose of the output layer is to output a vector of values that is in a format that is suitable for the type of problem that we're trying to address.
* This is handled using activation functions based on the problem.
1. Regression - Single output node for providing continous value.
2. Binary Classification - Sigmoid activation function in order to squishify values down to represent a probability.
3. Multi-class classification - Softmax function might have multiple output nodes in the output layer, one for each class that we're trying to predict.

### Neuron:
An artificial neuron is a mathematical function are elementary units in an artificial neural network. The artificial neuron receives one or more inputs and sums them to produce an output. Usually each input is separately weighted, and the sum is passed through a non-linear function known as an **activation  or transfer function**. The transfer functions can be sigmoid, rectified linear unit ReLU or step functions...

### Weight:
In computer science, synaptic weight refers to the strength or amplitude of a connection between two nodes. 

In a computational neural network, a vector or set of inputs ${\displaystyle {\textbf {x}}}$ and outputs ${\displaystyle {\textbf {y}}}$, or pre- and post-synaptic neurons respectively, are interconnected with synaptic weights represented by the matrix ${\displaystyle w}$, where for a linear neuron

![](https://wikimedia.org/api/rest_v1/media/math/render/svg/ed6616f818670502533e9f6181934324a77aafc7)

### Activation Function:

In artificial neural networks, the activation function of a node defines the output of that node, or "neuron," given an input or set of inputs. This output is then used as input for the next node and so on until a desired solution to the original problem is found.

It maps the resulting values into the desired range such as between 0 to 1 or -1 to 1 etc. (depending upon the choice of activation function). For example, the use of the logistic activation function would map all inputs in the real number domain into the range of 0 to 1.

### Node Map:

"Node Map" it's a visual diagram of the architecture or "topology" of our neural network. It's kind of like a flow chart in that it shows the path from inputs to outputs.

### Perceptron:

The first and simplest kind of neural network that we could talk about is the perceptron. A perceptron is just a single node or neuron of a neural network with nothing else. It can take any number of inputs and gives out an output. 

### Bias:

Bias is a constant which helps the model in a way that it can fit best for the given data.

In other words, Bias is a constant which gives freedom to perform best.

A bias value allows you to shift the activation function curve up or down.


## Inputs -> Outputs

### Explain the flow of information through a neural network from inputs to outputs. Be sure to include: inputs, weights, bias, and activation functions. How does it all flow from beginning to end?

Neuron is the basic building block of Neural Network. What a neuron does is it takes each of the input values, multplies each with corresponding weight, sums all of these products up, adds bias and then passes the sum through what is called an "activation function" the result of which is the final value.

There can be several layers of neurons and each layer can have one or more neurons. 

## Write your own perceptron code that can correctly classify a NAND gate. 

| x1 | x2 | y |
|----|----|---|
| 0  | 0  | 1 |
| 1  | 0  | 1 |
| 0  | 1  | 1 |
| 1  | 1  | 0 |

In [0]:
import numpy as np
np.random.seed(1)

inputs = np.array([[0,0,1],
                   [1,1,1],
                   [1,0,1],
                   [0,1,1]])
# Expected output for NAND (!AND)
correct_outputs = [[1],
                  [1],
                  [1],
                  [0]]

# Initial weights
weights = 2 * np.random.random((3,1)) - 1
# weights = np.random.random((3,1))

In [0]:
def sigmoid(x):
  return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
  return sigmoid(x) * (1 - sigmoid(x))

In [0]:
for iteration in range(10000):
  
  # Weighted sum of inputs and weights
  weighted_sum = np.dot(inputs, weights)
  
  # Activate with sigmoid function
  activated_output = sigmoid(weighted_sum)
  
  # Calculate Error
  error = correct_outputs - activated_output
  
  # Calculate weight adjustments with sigmoid_derivative
  adjustments = error * sigmoid_derivative(activated_output)
  
  # Update weights
  weights += np.dot(inputs.T, adjustments)
  
print('optimized weights after training: ')
print(weights)

print("Output After Training:")
print(activated_output)

optimized weights after training: 
[[ 13.31887341]
 [-12.91309088]
 [  6.48068717]]
Output After Training:
[[0.99846944]
 [0.99897943]
 [1.        ]
 [0.00160616]]


## Implement your own Perceptron Class and use it to classify a binary dataset like: 
- [The Pima Indians Diabetes dataset](https://raw.githubusercontent.com/ryanleeallred/datasets/master/diabetes.csv) 
- [Titanic](https://raw.githubusercontent.com/ryanleeallred/datasets/master/titanic.csv)
- [A two-class version of the Iris dataset](https://raw.githubusercontent.com/ryanleeallred/datasets/master/Iris.csv)

You may need to search for other's implementations in order to get inspiration for your own. There are *lots* of perceptron implementations on the internet with varying levels of sophistication and complexity. Whatever your approach, make sure you understand **every** line of your implementation and what its purpose is.

In [6]:
##### Your Code Here #####
# https://machinelearningmastery.com/implement-perceptron-algorithm-scratch-python/

from random import seed
from random import randrange
from csv import reader
 
# Load a CSV file
def load_csv(filename):
	dataset = list()
	with open(filename, 'r') as file:
		csv_reader = reader(file)
		for row in csv_reader:
			if not row:
				continue
			dataset.append(row)
	return dataset
 
# Convert string column to float
def str_column_to_float(dataset, column):
	for row in dataset:
		row[column] = float(row[column].strip())
 
# Convert string column to integer
def str_column_to_int(dataset, column):
	class_values = [row[column] for row in dataset]
	unique = set(class_values)
	lookup = dict()
	for i, value in enumerate(unique):
		lookup[value] = i
	for row in dataset:
		row[column] = lookup[row[column]]
	return lookup
 
# Split a dataset into k folds
def cross_validation_split(dataset, n_folds):
	dataset_split = list()
	dataset_copy = list(dataset)
	fold_size = int(len(dataset) / n_folds)
	for i in range(n_folds):
		fold = list()
		while len(fold) < fold_size:
			index = randrange(len(dataset_copy))
			fold.append(dataset_copy.pop(index))
		dataset_split.append(fold)
	return dataset_split
 
# Calculate accuracy percentage
def accuracy_metric(actual, predicted):
	correct = 0
	for i in range(len(actual)):
		if actual[i] == predicted[i]:
			correct += 1
	return correct / float(len(actual)) * 100.0
 
# Evaluate an algorithm using a cross validation split
def evaluate_algorithm(dataset, algorithm, n_folds, *args):
	folds = cross_validation_split(dataset, n_folds)
	scores = list()
	for fold in folds:
		train_set = list(folds)
		train_set.remove(fold)
		train_set = sum(train_set, [])
		test_set = list()
		for row in fold:
			row_copy = list(row)
			test_set.append(row_copy)
			row_copy[-1] = None
		predicted = algorithm(train_set, test_set, *args)
		actual = [row[-1] for row in fold]
		accuracy = accuracy_metric(actual, predicted)
		scores.append(accuracy)
	return scores
 
# Make a prediction with weights
def predict(row, weights):
	activation = weights[0]
	for i in range(len(row)-1):
		activation += weights[i + 1] * row[i]
	return 1.0 if activation >= 0.0 else 0.0
 
# Estimate Perceptron weights using stochastic gradient descent
def train_weights(train, l_rate, n_epoch):
	weights = [0.0 for i in range(len(train[0]))]
	for epoch in range(n_epoch):
		for row in train:
			prediction = predict(row, weights)
			error = row[-1] - prediction
			weights[0] = weights[0] + l_rate * error
			for i in range(len(row)-1):
				weights[i + 1] = weights[i + 1] + l_rate * error * row[i]
	return weights
 
# Perceptron Algorithm With Stochastic Gradient Descent
def perceptron(train, test, l_rate, n_epoch):
	predictions = list()
	weights = train_weights(train, l_rate, n_epoch)
	for row in test:
		prediction = predict(row, weights)
		predictions.append(prediction)
	return(predictions)
 
# Test the Perceptron algorithm on the sonar dataset
seed(1)
# load and prepare data
filename = 'diabetes.csv'
dataset = load_csv(filename)
for i in range(len(dataset[0])-1):
	str_column_to_float(dataset, i)
# convert string class to integers
str_column_to_int(dataset, len(dataset[0])-1)
# evaluate algorithm
n_folds = 5
l_rate = 0.01
n_epoch = 500
scores = evaluate_algorithm(dataset, perceptron, n_folds, l_rate, n_epoch)
print('Scores: %s' % scores)
print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))

Scores: [66.01307189542483, 64.70588235294117, 64.05228758169935, 64.70588235294117, 51.633986928104584]
Mean Accuracy: 62.222%


## Stretch Goals:

- Research "backpropagation" to learn how weights get updated in neural networks (tomorrow's lecture). 
- Implement a multi-layer perceptron. (for non-linearly separable classes)
- Try and implement your own backpropagation algorithm.
- What are the pros and cons of the different activation functions? How should you decide between them for the different layers of a neural network?